In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
import glob

warnings.filterwarnings("ignore")

In [2]:
def print_df(_df):
    print(_df.columns.tolist())
    first_row_values = _df.iloc[0].tolist()
    print(first_row_values)

In [6]:
file_paths = glob.glob('data/*.csv')


first_df = pd.read_csv(file_paths[0])  
other_dfs = [pd.read_csv(file_path, header=None) for file_path in file_paths[1:]]  


for df in other_dfs:
    df.columns = first_df.columns

uni_list = pd.concat([first_df] + other_dfs, ignore_index=True)


In [7]:
print_df(uni_list)

['프로그램구분', '기관', '일련번호', '지역', '국가명', '대학명(국문)', '대학명(영문)', '최소 학점', '어학성적', '특이사항', '참고사항', '수학가능학과/영어강의목록 등', 'Factsheet 여부', '교환학생수기 여부', '유의사항', '웹사이트 ']
['일반교환', '자매대학', 'E0008', '북미', '캐나다', '컨커디어대', 'Concordia University', '3.3', 'A-1', '* TOEIC/TOEFL ITP 제외\n* TOEFL iBT 각 영역별 20 이상\n※ English Literature 학과: \n   TOEFL iBT 110 이상 \n   Journalism and Communication\n   Studies 학과:  TOEFL iBT 100 이상\n* IELTS 각 영역 별 6.5 이상 필요\n* 토플 홈에디션 인정', '* 위치: Montreal, Quebec, Canada\n* 특징: Quebec 州 에 위치한 연구 중심 공립 대학 \n          학생 수 약 47,000명, 등록금 약 CAD 12,000/학기\n          캐나다 경영대학 순위 10위권 내 \n          #601-800 THE World University Rankings 2020', '★ 수학가능학과: Arts&Science, Fine Arts, The John Molson School of Business, Gina Cody School of Engineering and Computer Science\n* 첨부파일(Course List) 참조, 일부 수강제한있음\n\n★ 수강가능과목(2021): 국제팀 홈페이지_커뮤니티_자매대학소개_첨부 파일(Course List) 참조', 'Y(2020)\n/ COURSE LIST(2021)', 'Y(2013-2019)', '★ 2학기만 파견 가능 ', 'https://www.concordia.ca/international/visiting-exchange/st

In [8]:
uni_list.dropna(axis=0, how='all', inplace=True)
uni_list.dropna(axis=1, how='all', inplace=True)

In [9]:
uni_list['프로그램구분'].replace('교환학생\n +인턴십', '교환학생+인턴십', inplace=True)
uni_list['프로그램구분'].replace('순수', '일반교환', inplace=True)
uni_list['프로그램구분'].replace('방문', '방문교환', inplace=True)
uni_list['프로그램구분'].replace('특별', '방문교환', inplace=True)

In [ ]:
uni_count_result = uni_list.groupby(['파견학기', '모집차수'], as_index=False).count()[['파견학기', '모집차수', '일련번호']]
uni_count_result = uni_count_result.rename(columns={'일련번호': '파견가능 학교'})

In [ ]:
print_df(uni_list)

In [ ]:
uni_count_viz = uni_list.groupby(['파견학기', '모집차수'], as_index=False).count()[['파견학기', '모집차수', '일련번호']]
sns.barplot(x='파견학기', y='일련번호', hue='모집차수', data=uni_count_viz)
plt.title('파견가능 학교 수')
plt.ylabel('count')
plt.ylim(0, 500)

In [ ]:
uni_count_viz = uni_list[uni_list['프로그램구분'] == '일반교환'].groupby(['파견학기', '모집차수'], as_index=False).count()[['파견학기', '모집차수', '일련번호']]
sns.barplot(x='파견학기', y='일련번호', hue='모집차수', data=uni_count_viz)
plt.title('일반교환 파견가능 학교 수')
plt.ylabel('count')
plt.ylim(0, 400)

In [ ]:
sem_list = uni_list['파견학기'].unique()
normal_uni_list = uni_list[uni_list['프로그램구분'] == '일반교환']
pop_uni_list = normal_uni_list.iloc[:0].copy()
for s in sem_list:
    pop_uni_list = pd.concat([pop_uni_list, normal_uni_list[normal_uni_list['파견학기'] == s][normal_uni_list['모집차수'] == '1차'][~normal_uni_list['대학명(국문)'].isin(normal_uni_list[normal_uni_list['파견학기'] == s][normal_uni_list['모집차수'] == '2차']['대학명(국문)'].unique())]])

pop_uni_list = pop_uni_list[['국가명', '대학명(국문)', '대학명(영문)']]
pop_uni_list.drop_duplicates(subset=['대학명(국문)'], inplace=True)
pop_uni_list.sort_values(by=['국가명', '대학명(국문)'], inplace=True)

In [ ]:
result_list.dropna(axis=0, how='all', inplace=True)
result_list.dropna(axis=1, how='all', inplace=True)

In [ ]:
result = result_list.groupby(['파견학기', '모집차수'], as_index=False).count()[['파견학기', '모집차수', '대학']]
result = result.rename(columns={'대학': '에비합격자'})
target_sheet = doc.worksheet('학기 당 파견자')
set_with_dataframe(target_sheet, result)

In [ ]:
sns.barplot(x='파견학기', y='에비합격자', hue='모집차수', data=result)
plt.title('학기별 합격자 수')
plt.ylabel('count')

In [ ]:
scholarship_list = result_list[~result_list['장학'].isna()][['파견학기', '모집차수', '국가', '대학', '장학']]
target_sheet = doc.worksheet('장학')
set_with_dataframe(target_sheet, scholarship_list)